# Notebook for exercise 2

## 0. Get data ready

In [3]:
# Import the necessary libraries
import os   #helps me erase a warning that i am getting
import numpy as np  #the data are numpy when i first get them
from sklearn.preprocessing import StandardScaler  #need this for standardizing the data
from sklearn.decomposition import PCA #need this for the PCA in the data
import torch
from torch import nn #need this for my model (layers, loss functions, ...)
from sklearn.metrics import accuracy_score #need this to measure the accuracy of the model
import time  #need this to calcualate the training time
from pathlib import Path #need this for the save_model(). 
import matplotlib.pyplot as plt  # need this for the model's plots
import pandas as pd

In [4]:
os.environ["LOKY_MAX_CPU_COUNT"] = "8"  # This line erases a warning that i am getting

file_names = []
files = []
#create X_train, y_train, X_test, y_test
X_train = np.full((50000,3072),0,dtype=int)
X_test = np.full((10000,3072),0,dtype=int)
y_train = np.full((50000,),0,dtype=int)
y_test = np.full((10000,),0,dtype=int)

# the unpickle() function loads the CIFAR-10 data from the file
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Creating a list with the names of the files
for i in range(1,6):
    file = f"data_batch_{i}"
    file_names.append(file)
file_names.append("test_batch")

#storing the unpickled dictionaries to a list
for file in file_names:
    cifar10_dict = unpickle(file)
    files.append(cifar10_dict)

# Splitting the data to train and test set
# Printing the shapes to make sure that everything went well
i=0
for file in files:
    print(f"Append: {file[b'batch_label']}")
    if file[b'batch_label'] != b'testing batch 1 of 1':
        #this is the X_train, y_train
        X_train[i*10000:(i+1)*10000,:] = file[b'data']
        y_train[i*10000:(i+1)*10000] = file[b'labels'] 
        i+=1
       
    else:
        #i have just finished X_train, y_train   
        #this is the X_test, y_test
        X_test[:,:] = file[b'data']
        y_test[:] = file[b'labels']
        print(f"Shape X_train: {X_train.shape}\nShape y_train: {y_train.shape}")
        print(f"Shape X_test: {X_test.shape}\nShape y_test: {y_test.shape}\n")
        
        
    

Append: b'training batch 1 of 5'
Append: b'training batch 2 of 5'
Append: b'training batch 3 of 5'
Append: b'training batch 4 of 5'
Append: b'training batch 5 of 5'
Append: b'testing batch 1 of 1'
Shape X_train: (50000, 3072)
Shape y_train: (50000,)
Shape X_test: (10000, 3072)
Shape y_test: (10000,)



- The maximum margin classifier forms the basis of SVMs, searating classes with the widest posssible gap to ensure stability and robustness.
This stability under perturbations makes it a <u>**powerful choice**</u> for tasks where <u>**small changes in data are common**</u>

- SVMs are by definition two-class classifiers.

- In scenarios where the number of classes (c > 2), Support Vector Machines (SVMs) can be extended to handle multiclass classification using two common approaches:
1. One-vs-the-Rest Approach
Train one SVM model for each class. For class i, the SVM distinguishes between class i and all other classes.

2. One-vs-One Approach
Train c(c-1)/2 binary SVM models, each for a pair of classes (one-vs-one).
For a given input the predicted class is determined by **majority voting**, i.e., the class that receives the highest number of "votes" across all pairwise models is selected.


# AN DO OTI PAIRNIE POLY ORA NA TREJEI MPORO NA KRATISO MONO 2 CLASSES ETSI OSTE NA TREXEI PIO GRIGORA KAI NA DOKIIMAZO PERISSOTERA PRAGMATA